In [1]:
import torch
import torch_geometric
from torch_geometric.data import Data, DataLoader
import deepchem as dc
from rdkit import Chem
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data, InMemoryDataset
import random
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from torch.optim.lr_scheduler import ReduceLROnPlateau

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\deepchem\models\torch_models\__init__.py)


In [2]:
from torch_geometric.data import InMemoryDataset, Data

class DeepChemToPyGDataset(InMemoryDataset):
    def __init__(self, data_list, transform=None):
        """
        Converts a list of PyG `Data` objects into an InMemoryDataset.
        
        Args:
            data_list (list of Data): List of PyG Data objects.
            transform (callable, optional): Optional transform applied to each data object.
        """
        super(DeepChemToPyGDataset, self).__init__('.', transform, None, None)
        self.data, self.slices = self.collate(data_list)

    def get(self, idx):
        """
        Retrieves the data object at the specified index.
        """
        return super().get(idx)


In [3]:
pyg_dataset = torch.load('../../data/curated dataset/frag_pyg_dataset.pth')

C:\Users\sarab\AppData\Local\Temp\ipykernel_14204\3101895145.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pyg_dataset = torch.load('../../data/curated dataset/frag_py

In [4]:
node_features = pyg_dataset[0].x.shape[1]  # Should be 134 based on your description
edge_features = pyg_dataset[0].edge_attr.shape[1]  # Should be 6 based on your description


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, global_mean_pool
from torch_geometric.data import DataLoader

class PlanarFlow(nn.Module):
    def __init__(self, dim):
        super(PlanarFlow, self).__init__()
        self.u = nn.Parameter(torch.randn(dim))
        self.w = nn.Parameter(torch.randn(dim))
        self.b = nn.Parameter(torch.randn(1))

    def forward(self, z):
        linear_term = torch.sum(self.w * z, dim=1, keepdim=True) + self.b
        tanh_term = torch.tanh(linear_term)
        z_out = z + self.u * tanh_term
        psi = (1 - tanh_term**2) * self.w
        log_det = torch.log(torch.abs(1 + torch.sum(self.u * psi, dim=1)))
        return z_out, log_det

    def inverse(self, z):
        raise NotImplementedError("Planar flow inverse is not implemented")

class FlowGraphModel(nn.Module):
    def __init__(self, node_features, edge_features, hidden_dim, latent_dim, num_flows=4):
        super(FlowGraphModel, self).__init__()
        self.encoder = Encoder(node_features, edge_features, hidden_dim, latent_dim)
        self.flows = nn.ModuleList([PlanarFlow(latent_dim) for _ in range(num_flows)])
        self.decoder = Decoder(latent_dim, hidden_dim, node_features, edge_features)
        self.latent_dim = latent_dim

    def forward(self, x, edge_index, edge_attr, batch=None):
        z = self.encoder(x, edge_index, edge_attr, batch)
        log_det_sum = 0
        for flow in self.flows:
            z, log_det = flow(z)
            log_det_sum += log_det
        reconstructed_x, reconstructed_edge_attr = self.decoder(z, edge_index, batch)
        return reconstructed_x, reconstructed_edge_attr, z, log_det_sum

    def inverse(self, z, edge_index, batch=None):
        for flow in reversed(self.flows):
            z = flow.inverse(z)
        return self.decoder(z, edge_index, batch)

    def generate_multiple_outputs(self, x, edge_index, edge_attr, num_samples=5):
        outputs = []
        with torch.no_grad():
            # Encode the input
            z = self.encoder(x, edge_index, edge_attr)

            for _ in range(num_samples):
                # Sample a new latent vector by adding noise to the encoded vector
                z_sample = z + torch.randn_like(z) * 0.8

                # Apply flow transformations
                for flow in self.flows:
                    z_sample, _ = flow(z_sample)

                # Decode
                reconstructed_x, reconstructed_edge_attr = self.decoder(z_sample, edge_index)

                # Apply sigmoid to get probabilities
                reconstructed_x = torch.sigmoid(reconstructed_x)
                reconstructed_edge_attr = torch.sigmoid(reconstructed_edge_attr)

                # Threshold to get binary outputs
                reconstructed_x = (reconstructed_x > 0.5).float()
                reconstructed_edge_attr = (reconstructed_edge_attr > 0.5).float()

                outputs.append((reconstructed_x, reconstructed_edge_attr, edge_index))
        return outputs

class Encoder(nn.Module):
    def __init__(self, node_features, edge_features, hidden_dim, latent_dim):
        super(Encoder, self).__init__()
        self.conv1 = SAGEConv(node_features + edge_features, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim)
        self.conv3 = SAGEConv(hidden_dim, latent_dim)

    def forward(self, x, edge_index, edge_attr, batch=None):
        edge_attr_expanded = torch.zeros((x.size(0), edge_attr.size(1))).to(x.device)
        edge_attr_expanded[edge_index[0]] = edge_attr
        x = torch.cat([x, edge_attr_expanded], dim=1)

        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.conv3(x, edge_index)

        if batch is not None:
            x = global_mean_pool(x, batch)
        else:
            x = x.mean(dim=0, keepdim=True)

        return x

class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, node_features, edge_features):
        super(Decoder, self).__init__()
        self.fc = nn.Linear(latent_dim, hidden_dim)
        self.node_decoder = nn.Linear(hidden_dim, node_features)
        self.edge_decoder = nn.Linear(hidden_dim * 2, edge_features)

    def forward(self, z, edge_index, batch=None):
        h = self.fc(z)
        if batch is not None:
            h = h[batch]
        else:
            h = h.repeat(edge_index.max().item() + 1, 1)

        reconstructed_x = self.node_decoder(h)

        row, col = edge_index
        edge_h = torch.cat([h[row], h[col]], dim=1)
        reconstructed_edge_attr = self.edge_decoder(edge_h)

        return reconstructed_x, reconstructed_edge_attr


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#model = model.to(device)

In [7]:
from torch_geometric.loader import DataLoader
import torch.nn.functional as F

def train_flow_model_with_early_stopping(
    model, train_dataset, val_dataset, num_epochs, batch_size, lr, device, patience, latent_dim
):
    """
    Trains a flow-based model with early stopping.

    Parameters:
        model (torch.nn.Module): The flow-based model to train.
        train_dataset (Dataset): Training dataset.
        val_dataset (Dataset): Validation dataset.
        num_epochs (int): Maximum number of epochs to train.
        batch_size (int): Batch size for training.
        lr (float): Learning rate for the optimizer.
        device (torch.device): Device to use ('cpu' or 'cuda').
        patience (int): Number of epochs to wait for improvement before stopping.
        latent_dim (int): Dimensionality of the latent space (used for flow loss normalization).

    Returns:
        torch.nn.Module: Trained model with the best weights (based on validation loss).
    """
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    best_loss = float("inf")  # Best validation loss observed so far
    patience_counter = 0      # Counts epochs without improvement
    best_model_state = None   # To store the best model weights

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        total_train_loss = 0
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()

            # Forward pass
            reconstructed_x, reconstructed_edge_attr, z, log_det = model(
                batch.x, batch.edge_index, batch.edge_attr, batch.batch
            )

            # Reconstruction loss
            recon_loss = F.mse_loss(reconstructed_x, batch.x) + F.mse_loss(reconstructed_edge_attr, batch.edge_attr)

            # Flow loss (negative log-likelihood), normalized by latent_dim
            flow_loss = -torch.mean(log_det) / latent_dim

            # Total loss
            loss = 0.5 * recon_loss + flow_loss

            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)

        # Validation phase
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                reconstructed_x, reconstructed_edge_attr, z, log_det = model(
                    batch.x, batch.edge_index, batch.edge_attr, batch.batch
                )

                # Validation loss
                recon_loss = F.mse_loss(reconstructed_x, batch.x) + F.mse_loss(reconstructed_edge_attr, batch.edge_attr)
                flow_loss = -torch.mean(log_det) / latent_dim
                loss = 0.5 * recon_loss + flow_loss
                total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_loader)

        # Logging
        print(f"Epoch [{epoch+1}/{num_epochs}]")
        print(f"  Train Loss: {avg_train_loss:.4f}")
        print(f"  Val Loss: {avg_val_loss:.4f}")

        # Early stopping check
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            patience_counter = 0
            best_model_state = model.state_dict()  # Save the best model state
            print(f"  New best model saved with validation loss: {best_loss:.4f}")
        else:
            patience_counter += 1
            print(f"  No improvement. Patience: {patience_counter}/{patience}")

        if patience_counter >= patience:
            print(f"Early stopping triggered at epoch {epoch+1}.")
            break

    # Restore the best model state
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print("Best model weights restored.")

    return model


In [8]:
import optuna
import torch
import torch.nn.functional as F
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
from torch_geometric.loader import DataLoader as PyGDataLoader
from multiprocessing import cpu_count

c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:


# Define the loss function
def loss_function(reconstructed_x, x, reconstructed_edge_attr, edge_attr, log_det, latent_dim):
    """Custom loss function combining reconstruction and flow losses."""
    recon_loss = F.mse_loss(reconstructed_x, x) + F.mse_loss(reconstructed_edge_attr, edge_attr)
    flow_loss = -torch.mean(log_det) / latent_dim  # Normalize by latent_dim
    return 0.5 * recon_loss + flow_loss

# Define the training function with early stopping
def train_flow_model_with_early_stopping(
    model, train_loader, val_loader, num_epochs, lr, device, patience, latent_dim, trial_number
):
    """
    Trains the FlowGraphModel with early stopping and logs progress including trial number.
    
    Parameters:
        model (nn.Module): The flow-based model to train.
        train_loader (DataLoader): DataLoader for the training dataset.
        val_loader (DataLoader): DataLoader for the validation dataset.
        num_epochs (int): Maximum number of epochs to train.
        lr (float): Learning rate.
        device (torch.device): Device ('cpu' or 'cuda') to use for training.
        patience (int): Number of epochs to wait for improvement before stopping.
        latent_dim (int): Dimensionality of the latent space for flow normalization.
        trial_number (int): Current trial number from Optuna.

    Returns:
        float: The best validation loss.
    """
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=5, verbose=False)

    best_val_loss = float("inf")
    patience_counter = 0
    best_model_state = None

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        total_train_loss = 0
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()

            # Forward pass
            reconstructed_x, reconstructed_edge_attr, _, log_det = model(
                batch.x, batch.edge_index, batch.edge_attr, batch.batch
            )

            # Compute loss
            loss = loss_function(
                reconstructed_x, batch.x, reconstructed_edge_attr, batch.edge_attr, log_det, latent_dim
            )
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)

        # Validation phase
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                reconstructed_x, reconstructed_edge_attr, _, log_det = model(
                    batch.x, batch.edge_index, batch.edge_attr, batch.batch
                )
                loss = loss_function(
                    reconstructed_x, batch.x, reconstructed_edge_attr, batch.edge_attr, log_det, latent_dim
                )
                total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_loader)

        # Logging
        print(
            f"[Trial {trial_number}] Epoch [{epoch+1}/{num_epochs}] - "
            f"Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}"
        )

        # Scheduler step
        scheduler.step(avg_val_loss)

        # Early stopping check
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            best_model_state = model.state_dict()  # Save the best model state
            print(f"[Trial {trial_number}] New best model saved with validation loss: {best_val_loss:.4f}.")
        else:
            patience_counter += 1
            print(f"[Trial {trial_number}] No improvement. Patience counter: {patience_counter}/{patience}.")

        if patience_counter >= patience:
            print(f"[Trial {trial_number}] Early stopping triggered.")
            break

    # Restore the best model state
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"[Trial {trial_number}] Best model state restored.")
    return best_val_loss


# Define the objective function for Optuna
def objective(trial):
    # Hyperparameters to tune
    hidden_dim = trial.suggest_int("hidden_dim", 64, 512, step=64)
    latent_dim = trial.suggest_int("latent_dim", 32, 128, step=32)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32, 64])
    num_flows = trial.suggest_int("num_flows", 1, 10)
    patience = trial.suggest_int("patience", 3, 10)

    # Log the trial number and parameters
    print(f"\n[Trial {trial.number}] Starting with parameters:")
    print(f"  HIDDEN_DIM: {hidden_dim}")
    print(f"  LATENT_DIM: {latent_dim}")
    print(f"  LEARNING_RATE: {learning_rate:.5f}")
    print(f"  BATCH_SIZE: {batch_size}")
    print(f"  NUM_FLOWS: {num_flows}")
    print(f"  PATIENCE: {patience}")

    # Split dataset
    train_ratio = 0.8
    num_train = int(len(pyg_dataset) * train_ratio)
    num_valid = len(pyg_dataset) - num_train
    train_dataset, valid_dataset = random_split(pyg_dataset, [num_train, num_valid])

    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    # Initialize the model
    model = FlowGraphModel(node_features, edge_features, hidden_dim, latent_dim, num_flows)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Train the model
    val_loss = train_flow_model_with_early_stopping(
        model, train_loader, val_loader, num_epochs=60, lr=learning_rate, device=device,
        patience=patience, latent_dim=latent_dim, trial_number=trial.number
    )

    return val_loss


In [10]:
# Run Optuna study
def run_hyperparameter_tuning():
    # Set multiprocessing
    n_jobs = min(torch.multiprocessing.cpu_count(), 10)  # Use up to 4 processes
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=200, n_jobs=n_jobs)  # Parallel execution

    # Print the best trial
    best_trial = study.best_trial
    print("\nBest Trial:")
    print(f"  Number: {best_trial.number}")
    print(f"  Params: {best_trial.params}")
    print(f"  Validation Loss: {best_trial.value:.4f}")

    # Save study results
    study.trials_dataframe().to_csv("optuna_study_results.csv", index=False)
    return study

# Example usage
if __name__ == "__main__":
    # Ensure all global variables are defined (e.g., pyg_dataset, node_features, edge_features)
    study = run_hyperparameter_tuning()

[I 2024-12-09 11:10:52,856] A new study created in memory with name: no-name-60a1e404-0395-4199-9a8a-913a9d60da9d



[Trial 0] Starting with parameters:
  HIDDEN_DIM: 512
  LATENT_DIM: 64
  LEARNING_RATE: 0.07962
  BATCH_SIZE: 32
  NUM_FLOWS: 6
  PATIENCE: 3

[Trial 1] Starting with parameters:
  HIDDEN_DIM: 64
  LATENT_DIM: 96
  LEARNING_RATE: 0.00257
  BATCH_SIZE: 32
  NUM_FLOWS: 8
  PATIENCE: 3

[Trial 2] Starting with parameters:
  HIDDEN_DIM: 384
  LATENT_DIM: 96
  LEARNING_RATE: 0.00114
  BATCH_SIZE: 64
  NUM_FLOWS: 1
  PATIENCE: 6

[Trial 3] Starting with parameters:
  HIDDEN_DIM: 320
  LATENT_DIM: 128
  LEARNING_RATE: 0.00003
  BATCH_SIZE: 8
  NUM_FLOWS: 1
  PATIENCE: 9

[Trial 4] Starting with parameters:
  HIDDEN_DIM: 448
  LATENT_DIM: 96
  LEARNING_RATE: 0.00035
  BATCH_SIZE: 8
  NUM_FLOWS: 4
  PATIENCE: 7

[Trial 5] Starting with parameters:
  HIDDEN_DIM: 320
  LATENT_DIM: 128
  LEARNING_RATE: 0.00002
  BATCH_SIZE: 32
  NUM_FLOWS: 3
  PATIENCE: 5

[Trial 6] Starting with parameters:
  HIDDEN_DIM: 320
  LATENT_DIM: 32
  LEARNING_RATE: 0.00002
  BATCH_SIZE: 16
  NUM_FLOWS: 6
  PATIENCE: 3


c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


[Trial 2] Epoch [1/60] - Train Loss: 0.0748, Val Loss: 0.0459
[Trial 2] New best model saved with validation loss: 0.0459.
[Trial 5] Epoch [1/60] - Train Loss: 0.1034, Val Loss: 0.0513
[Trial 5] New best model saved with validation loss: 0.0513.
[Trial 2] Epoch [2/60] - Train Loss: 0.0447, Val Loss: 0.0447
[Trial 2] New best model saved with validation loss: 0.0447.
[Trial 1] Epoch [1/60] - Train Loss: 0.0811, Val Loss: 0.0500
[Trial 1] New best model saved with validation loss: 0.0500.
[Trial 0] Epoch [1/60] - Train Loss: 5757396.6640, Val Loss: 439.6751
[Trial 0] New best model saved with validation loss: 439.6751.
[Trial 2] Epoch [3/60] - Train Loss: 0.0443, Val Loss: 0.0445
[Trial 2] New best model saved with validation loss: 0.0445.
[Trial 8] Epoch [1/60] - Train Loss: 8.9123, Val Loss: 0.0631
[Trial 8] New best model saved with validation loss: 0.0631.
[Trial 5] Epoch [2/60] - Train Loss: 0.0476, Val Loss: 0.0460
[Trial 5] New best model saved with validation loss: 0.0460.
[Trial

[I 2024-12-09 11:14:09,914] Trial 8 finished with value: 0.0631121148665746 and parameters: {'hidden_dim': 64, 'latent_dim': 96, 'learning_rate': 0.04703980056207048, 'batch_size': 16, 'num_flows': 5, 'patience': 4}. Best is trial 8 with value: 0.0631121148665746.


...

[Trial 196] Epoch [60/60] - Train Loss: -0.5734, Val Loss: -0.5741
[Trial 196] New best model saved with validation loss: -0.5741.
[Trial 196] Best model state restored.
[Trial 198] Epoch [54/60] - Train Loss: -0.6133, Val Loss: -0.6179
[Trial 198] New best model saved with validation loss: -0.6179.
[Trial 197] Epoch [54/60] - Train Loss: -0.1207, Val Loss: -0.1214
[Trial 197] New best model saved with validation loss: -0.1214.
[Trial 198] Epoch [55/60] - Train Loss: -0.6204, Val Loss: -0.6242
[Trial 198] New best model saved with validation loss: -0.6242.
[Trial 197] Epoch [55/60] - Train Loss: -0.1212, Val Loss: -0.1220
[Trial 197] New best model saved with validation loss: -0.1220.
[Trial 198] Epoch [56/60] - Train Loss: -0.6275, Val Loss: -0.6319
[Trial 198] New best model saved with validation loss: -0.6319.
[Trial 197] Epoch [56/60] - Train Loss: -0.1218, Val Loss: -0.1226
[Trial 197] New best model saved with validation loss: -0.1226.
[Trial 198] Epoch [57/60] - Train Loss: -0.6

[I 2024-12-09 22:43:00,411] Trial 198 finished with value: -0.6484659597277641 and parameters: {'hidden_dim': 256, 'latent_dim': 32, 'learning_rate': 0.00019152468291034342, 'batch_size': 8, 'num_flows': 5, 'patience': 4}. Best is trial 186 with value: -0.906932536760966.


[Trial 198] Epoch [60/60] - Train Loss: -0.6474, Val Loss: -0.6485
[Trial 198] New best model saved with validation loss: -0.6485.
[Trial 198] Best model state restored.


[I 2024-12-09 22:43:04,603] Trial 197 finished with value: -0.1592143341898918 and parameters: {'hidden_dim': 384, 'latent_dim': 32, 'learning_rate': 0.00014607790805251409, 'batch_size': 8, 'num_flows': 5, 'patience': 4}. Best is trial 186 with value: -0.906932536760966.


[Trial 197] Epoch [60/60] - Train Loss: -0.1400, Val Loss: -0.1592
[Trial 197] New best model saved with validation loss: -0.1592.
[Trial 197] Best model state restored.

Best Trial:
  Number: 186
  Params: {'hidden_dim': 384, 'latent_dim': 32, 'learning_rate': 0.0001448435077569811, 'batch_size': 8, 'num_flows': 7, 'patience': 4}
  Validation Loss: -0.9069
